# Extracting CIGALE fluxes

[CIGALE](https://cigale.lam.fr/) is an SED fitting code relying on the energy balance principle. We use it to make a Bayesian posterior estimate of the broadband fluxes. This is done by fitting library of model SEDs (the prior). The Bayesian posteriors result in a realistic SED. We do this for both the UV-MIR fluxes (named "shortbay"), and the UV-FIR fluxes ("fullbay").

The process of the SED fitting might depends on the version of CIGALE. We used version 0.12.0, which includes the THEMIS dust emission model. More recent versions allow you to specify a distance to the galaxy even for nearby galaxies (were redshift is not an appropriate distance metric), and hence we recommend using the most recent version.

The (input and output) data that we used for CIGALE can be found in `./data/CIGALE/`. We processed the data files to only include the relevant galaxies (throwing away the galaxies that have a $\chi_r^2 > 10$), and use space-separated csv files.

Note that for the UV-MIR SED fit, you need to add a "fake" galaxy that has no missing values, or else all bands that contain only nans (the FIR bands) will be discarded. All CIGALE fluxes are in mJy.

In [17]:
import os
import numpy as np
import pandas as pd
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u

pd.options.display.max_columns = 99

In [ ]:
os.chdir('..')  # change to root directory

In [2]:
def head(df):
    print('Shape', df.shape)
    return df.head()

The redshift metadata should be present in `./data/redshift.pkl`, connecting the galaxy id to its redshift. The redshift is corrected for proper motion in the case of dustpedia, so we can use it as a distance measure.

In [3]:
# Redshift metadata (for H-ATLAS + DustPedia)
import pickle
with open('./data/redshift.pkl', 'rb') as infile:
    z = pickle.load(infile)
    
z.head()

G15.DR1.7961     0.08373
G12.DR1.1506     0.06278
G09.DR1.7588     0.08147
G15.DR1.20104    0.09766
G15.DR1.5224     0.05311
Name: redshift, dtype: float64

In [7]:
# From CIGALE broadband name to our standard broadband names
colmap = {'FUV': 'GALEX_FUV', 'NUV': 'GALEX_NUV', 'u_prime': 'SDSS_u', 'g_prime': 'SDSS_g', 'r_prime': 'SDSS_r',
          'i_prime': 'SDSS_i', 'z_prime': 'SDSS_z', 'J_2mass': '2MASS_J', 'H_2mass': '2MASS_H', 
          'Ks_2mass': '2MASS_Ks', 'WISE1': 'WISE_3.4', 'WISE2': 'WISE_4.6', 'WISE3': 'WISE_12', 'WISE4': 'WISE_22',
          'PACS_blue': 'PACS_70', 'PACS_green': 'PACS_100', 'PACS_red': 'PACS_160', 'PSW_HIPE': 'SPIRE_250',
          'PMW_HIPE': 'SPIRE_350', 'PLW_HIPE': 'SPIRE_500'}
fluxmap = {'bayes.'+k: v for k, v in colmap.items()}
errmap = {'bayes.'+k+'_err': v for k, v in colmap.items()}

# Example for dustpedia full

In [19]:
df_raw = pd.read_csv('./data/CIGALE/dustpedia/full/out/results.txt', sep=' ', index_col=0)
idx_dp = df_raw.index
head(df_raw)

Shape (715, 160)


,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.FUV,bayes.FUV_err,bayes.HFI1,bayes.HFI1_err,bayes.HFI2,bayes.HFI2_err,bayes.HFI3,bayes.HFI3_err,bayes.H_2mass,bayes.H_2mass_err,bayes.IRAC1,bayes.IRAC1_err,bayes.IRAC2,bayes.IRAC2_err,bayes.IRAC3,bayes.IRAC3_err,bayes.IRAC4,bayes.IRAC4_err,bayes.IRAS3,bayes.IRAS3_err,bayes.IRAS4,bayes.IRAS4_err,bayes.J_2mass,bayes.J_2mass_err,bayes.Ks_2mass,bayes.Ks_2mass_err,bayes.MIPS1,bayes.MIPS1_err,bayes.MIPS2,bayes.MIPS2_err,bayes.MIPS3,bayes.MIPS3_err,bayes.NUV,bayes.NUV_err,bayes.PACS_blue,bayes.PACS_blue_err,bayes.PACS_green,bayes.PACS_green_err,bayes.PACS_red,bayes.PACS_red_err,bayes.PLW_HIPE,bayes.PLW_HIPE_err,bayes.PMW_HIPE,bayes.PMW_HIPE_err,bayes.PSW_HIPE,bayes.PSW_HIPE_err,bayes.WISE1,...,best.sfh.sfr100Myrs,best.sfh.sfr10Myrs,best.stellar.lum,best.stellar.lum_ly,best.stellar.lum_ly_old,best.stellar.lum_ly_young,best.stellar.lum_old,best.stellar.lum_young,best.stellar.m_gas,best.stellar.m_gas_old,best.stellar.m_gas_young,best.stellar.m_star,best.stellar.m_star_old,best.stellar.m_star_young,best.stellar.n_ly,best.stellar.n_ly_old,best.stellar.n_ly_young,best.FUV,best.NUV,best.u_prime,best.g_prime,best.r_prime,best.i_prime,best.z_prime,best.J_2mass,best.H_2mass,best.Ks_2mass,best.WISE1,best.WISE2,best.WISE3,best.WISE4,best.IRAC1,best.IRAC2,best.IRAC3,best.IRAC4,best.MIPS1,best.MIPS2,best.MIPS3,best.PACS_blue,best.PACS_green,best.PACS_red,best.PSW_HIPE,best.PMW_HIPE,best.PLW_HIPE,best.HFI1,best.HFI2,best.HFI3,best.IRAS3,best.IRAS4
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NGC5358,9.691433e-16,6.473464e-16,0.087310,0.033375,23.866408,31.182455,9.998848,13.778548,3.269103,4.648377,42.183849,0.734607,16.258697,0.278535,10.641350,0.158677,7.488449,0.331544,5.334132,1.013181,5.868554,6.921993,18.683654,22.280756,34.745128,0.531463,33.374730,0.573589,1.836273,1.209321,8.393235,10.980411,35.412189,41.519855,0.177670,0.023565,8.219531,10.988983,19.394848,22.940595,35.413769,41.396937,11.718560,16.017501,23.717610,31.009033,36.390517,45.057772,17.328304,...,9.685645e-16,9.685645e-16,1.803844e+23,6.440348e+20,3.669400e+20,2.770948e+20,1.789273e+23,1.457174e+21,0.000276,0.000276,2.938641e-10,0.000689,0.000689,9.391781e-09,1.626926e+38,7.860879e+37,8.408380e+37,0.079967,0.171561,1.917633,8.640379,15.625352,20.958734,27.308052,34.670595,42.079884,33.349126,17.360985,10.223888,2.778924,1.750736,16.244587,10.681357,7.318116,4.680366,1.821670,6.054239,82.662284,5.779339,20.969618,85.962020,136.645007,105.699672,58.084695,106.169032,50.434149,17.603373,4.666016,20.180743
NGC3252,6.324510e-14,1.909571e-14,1.264838,0.240478,1079.445250,144.697362,350.101720,53.727284,97.802839,16.183423,71.557526,4.120736,36.979430,1.203608,22.691926,0.785959,34.478601,3.860660,77.320777,13.807533,622.447434,122.697276,2633.676846,365.997177,59.091333,3.593591,58.583446,3.164284,96.791586,13.126792,1115.922909,221.345480,3559.037602,342.565863,2.108916,0.186100,1106.770024,223.001881,2722.049105,375.087447,3469.128572,332.975229,426.698180,64.166211,1069.185936,143.567563,2177.470041,249.401097,38.202029,...,8.065522e-14,8.065522e-14,5.198644e+23,2.376435e+22,6.898527e+20,2.307450e+22,3.985212e+23,1.213432e+23,0.000537,0.000537,2.447093e-08,0.001307,0.001307,7.820812e-07,7.162661e+39,1.607553e+38,7.001906e+39,1.353288,2.058061,5.216361,17.086438,27.832746,36.209836,46.992972,59.866644,73.156663,59.282115,38.478497,21.984954,76.871900,92.482947,37.486061,22.614321,35.744012,81.760514,99.107897,1095.020220,3600.927788,1084.119264,2729.057353,3509.481143,2207.891549,1083.201410,431.783076,1093.613433,354.188243,98.879911,612.325681,2637.281134
ESO407-002,1.641142e-14,9.844825e-15,0.380446,0.125960,382.392122,41.493289,119.585332,14.940619,32.689790,4.426000,46.984543,3.395084,23.766008,1.059751,14.671787,0.689558,20.958344,1.731806,44.986879,6.288138,324.496794,55.883152,1226.522419,153.261277,38.719864,2.590420,38.536199,2.634053,44.860837,5.943450,579.490557,99.126718,14

In [11]:
# Bayesian fluxes in mJy
df_fluxbay = df_raw[fluxmap.keys()].rename(columns=fluxmap)
head(df_fluxbay)

Shape (715, 20)


,GALEX_FUV,GALEX_NUV,SDSS_u,SDSS_g,SDSS_r,SDSS_i,SDSS_z,2MASS_J,2MASS_H,2MASS_Ks,WISE_3.4,WISE_4.6,WISE_12,WISE_22,PACS_70,PACS_100,PACS_160,SPIRE_250,SPIRE_350,SPIRE_500
id,,,,,,,,,,,,,,,,,,,,
NGC5358,0.087310,0.177670,1.850601,8.480688,15.454394,20.836358,27.302727,34.745128,42.183849,33.374730,17.328304,10.187100,3.272222,1.818025,8.219531,19.394848,35.413769,36.390517,23.717610,11.718560
NGC3252,1.264838,2.108916,5.960151,19.823463,30.226618,38.058639,47.678155,59.091333,71.557526,58.583446,38.202029,22.046256,72.810722,90.156540,1106.770024,2722.049105,3469.128572,2177.470041,1069.185936,426.698180
ESO407-002,0.380446,0.811733,3.279018,12.458522,19.494781,24.917328,31.161413,38.719864,46.984543,38.536199,24.668771,14.241665,39.880300,41.966934,577.942455,1262.853863,1414.810439,805.764186,378.585178,146.559760
NGC4194,2.295307,5.129439,12.199824,32.685684,48.524164,61.728676,79.648964,101.646181,124.571946,106.957601,92.464885,87.085491,942.912466,3060.277866,28561.772867,27881.937484,14312.772257,4833.525812,1820.938763,609.602403
NGC7172,0.431111,0.977902,6.244895,35.005812,77.316319,125.538816,190.042305,299.039677,420.034380,369.281104,243.698001,142.788369,397.213328,542.513404,7719.235322,13603.526643,12377.054018,6062.733527,2664.090018,984.871415


In [24]:
# Same for Bayesian uncertainties in mJy
df_errbay = df_raw[errmap.keys()].rename(columns=errmap)

In [20]:
dist_lum = cosmo.luminosity_distance(z.loc[idx_dp])
dist_lum

<Quantity [40.91693096, 20.25902475, 24.01185896, 39.28263305, 34.02805246,
            3.20141983,  4.6073195 , 17.15487076,  4.35943301, 26.00427255,
           17.0758066 , 26.58650883, 19.02304174, 26.73615746, 28.3936469 ,
           32.62375699, 33.85513371, 21.63587103, 14.00076867, 10.86219533,
           34.11302373, 28.26256853, 19.726267  , 28.79062208, 13.95890674,
           15.7864758 , 10.27717267, 67.65371467, 10.71287197, 19.43376865,
           21.11945083, 23.74793041,  3.84938444,  7.33928225, 27.61620914,
           13.06418096,  9.90479525, 13.87209595, 21.51351241, 19.79629393,
           17.0758066 ,  6.57045686, 30.15444472, 35.13329967, 37.94780124,
           19.34418819, 13.8767602 , 35.79041423, 25.20714185, 17.98860263,
           19.98010432,  9.50200405, 12.70736307, 30.29440883, 30.59985554,
           25.76480046, 19.79629393, 14.73013213, 37.5637701 , 13.76723507,
           14.66227112, 23.38030528,  7.51422815, 13.86306933, 29.14070353,
           4

Flux (in mJy) to luminosity (in W/Hz), without K-correction (see e.g. Hogg 2000).

$$L_{(1+z)\nu} = \frac{4\pi D_L^2 S_\nu}{1+z}$$  

In [21]:
# Everything to SI units

mult_fact = 4*np.pi*np.square(dist_lum.to(u.m).value)*1e-29/(1+z.loc[idx_dp])
mult_fact.head()

id
NGC5358       1.984943e+20
NGC3252       4.888453e+19
ESO407-002    6.861550e+19
NGC4194       1.830206e+20
NGC7172       1.374922e+20
Name: redshift, dtype: float64

In [22]:
df_lumbay = df_fluxbay.multiply(mult_fact, axis=0)
head(df_lumbay)

Shape (715, 20)


,GALEX_FUV,GALEX_NUV,SDSS_u,SDSS_g,SDSS_r,SDSS_i,SDSS_z,2MASS_J,2MASS_H,2MASS_Ks,WISE_3.4,WISE_4.6,WISE_12,WISE_22,PACS_70,PACS_100,PACS_160,SPIRE_250,SPIRE_350,SPIRE_500
id,,,,,,,,,,,,,,,,,,,,
NGC5358,1.733052e+19,3.526650e+19,3.673337e+20,1.683368e+21,3.067609e+21,4.135898e+21,5.419436e+21,6.896710e+21,8.373253e+21,6.624694e+21,3.439570e+21,2.022081e+21,6.495175e+20,3.608676e+20,1.631530e+21,3.849767e+21,7.029431e+21,7.223310e+21,4.707810e+21,2.326067e+21
NGC3252,6.183103e+19,1.030934e+20,2.913592e+20,9.690606e+20,1.477614e+21,1.860479e+21,2.330724e+21,2.888652e+21,3.498056e+21,2.863824e+21,1.867488e+21,1.077721e+21,3.559318e+21,4.407260e+21,5.410393e+22,1.330661e+23,1.695867e+23,1.064446e+23,5.226665e+22,2.085894e+22
ESO407-002,2.610452e+19,5.569750e+19,2.249914e+20,8.548477e+20,1.337644e+21,1.709715e+21,2.138156e+21,2.656783e+21,3.223868e+21,2.644181e+21,1.692660e+21,9.771990e+20,2.736407e+21,2.879582e+21,3.965581e+22,8.665135e+22,9.707793e+22,5.528791e+22,2.597681e+22,1.005627e+22
NGC4194,4.200884e+20,9.387929e+20,2.232819e+21,5.982152e+21,8.880920e+21,1.129762e+22,1.457740e+22,1.860334e+22,2.279923e+22,1.957544e+22,1.692298e+22,1.593844e+22,1.725724e+23,5.600938e+23,5.227392e+24,5.102968e+24,2.619532e+24,8.846346e+23,3.332692e+23,1.115698e+23
NGC7172,5.927441e+19,1.344538e+20,8.586243e+20,4.813025e+21,1.063039e+22,1.726060e+22,2.612933e+22,4.111562e+22,5.775144e+22,5.077326e+22,3.350657e+22,1.963228e+22,5.461372e+22,7.459135e+22,1.061334e+24,1.870378e+24,1.701748e+24,8.335784e+23,3.662915e+23,1.354121e+23


In [25]:
df_lumerrbay = df_errbay.multiply(mult_fact, axis=0)

# Streamline into function

In [36]:
d_data = {}  # Dictionary of dataframes/series

def process_CIGALE_data_sim(dataset, simname):
    """Read CIGALE output for particular simulation and dataset"""
    
    df_raw = pd.read_csv(f'./data/CIGALE/{dataset}/{simname}/out/results.txt', sep=' ', index_col=0)
    idx = df_raw.index
    # Rename
    df_fluxbay = df_raw[fluxmap.keys()].rename(columns=fluxmap)
    df_bayerr = df_raw[errmap.keys()].rename(columns=errmap)
    # Convert from mJy to W/Hz
    dist_lum = cosmo.luminosity_distance(z.loc[idx])
    mult_fact = 4*np.pi*np.square(dist_lum.to(u.m).value)*1e-29/(1+z.loc[idx])
    df_lumbay = df_fluxbay.multiply(mult_fact, axis=0)
    df_lumbayerr = df_bayerr.multiply(mult_fact, axis=0)
    return df_lumbay, df_lumbayerr

def process_CIGALE_sim(simname):
    """Read CIGALE output and put in d_dfs"""
    
    df_lumbay_dp, df_lumbayerr_dp = process_CIGALE_data_sim('dustpedia', simname)
    df_lumbay_ha, df_lumbayerr_ha = process_CIGALE_data_sim('hatlas', simname)
    df_lumbay = pd.concat([df_lumbay_dp, df_lumbay_ha], axis=0)
    df_lumbayerr = pd.concat([df_lumbayerr_dp, df_lumbayerr_ha], axis=0)
    d_data[f'{simname}bay'] = df_lumbay
    d_data[f'{simname}bayerr'] = df_lumbayerr

In [37]:
process_CIGALE_sim('full')
process_CIGALE_sim('short')

In [38]:
d_data.keys()

dict_keys(['fullbay', 'fullbayerr', 'shortbay', 'shortbayerr'])

In [39]:
head(d_data['fullbay'])

Shape (4333, 20)


,GALEX_FUV,GALEX_NUV,SDSS_u,SDSS_g,SDSS_r,SDSS_i,SDSS_z,2MASS_J,2MASS_H,2MASS_Ks,WISE_3.4,WISE_4.6,WISE_12,WISE_22,PACS_70,PACS_100,PACS_160,SPIRE_250,SPIRE_350,SPIRE_500
id,,,,,,,,,,,,,,,,,,,,
NGC5358,1.733052e+19,3.526650e+19,3.673337e+20,1.683368e+21,3.067609e+21,4.135898e+21,5.419436e+21,6.896710e+21,8.373253e+21,6.624694e+21,3.439570e+21,2.022081e+21,6.495175e+20,3.608676e+20,1.631530e+21,3.849767e+21,7.029431e+21,7.223310e+21,4.707810e+21,2.326067e+21
NGC3252,6.183103e+19,1.030934e+20,2.913592e+20,9.690606e+20,1.477614e+21,1.860479e+21,2.330724e+21,2.888652e+21,3.498056e+21,2.863824e+21,1.867488e+21,1.077721e+21,3.559318e+21,4.407260e+21,5.410393e+22,1.330661e+23,1.695867e+23,1.064446e+23,5.226665e+22,2.085894e+22
ESO407-002,2.610452e+19,5.569750e+19,2.249914e+20,8.548477e+20,1.337644e+21,1.709715e+21,2.138156e+21,2.656783e+21,3.223868e+21,2.644181e+21,1.692660e+21,9.771990e+20,2.736407e+21,2.879582e+21,3.965581e+22,8.665135e+22,9.707793e+22,5.528791e+22,2.597681e+22,1.005627e+22
NGC4194,4.200884e+20,9.387929e+20,2.232819e+21,5.982152e+21,8.880920e+21,1.129762e+22,1.457740e+22,1.860334e+22,2.279923e+22,1.957544e+22,1.692298e+22,1.593844e+22,1.725724e+23,5.600938e+23,5.227392e+24,5.102968e+24,2.619532e+24,8.846346e+23,3.332692e+23,1.115698e+23
NGC7172,5.927441e+19,1.344538e+20,8.586243e+20,4.813025e+21,1.063039e+22,1.726060e+22,2.612933e+22,4.111562e+22,5.775144e+22,5.077326e+22,3.350657e+22,1.963228e+22,5.461372e+22,7.459135e+22,1.061334e+24,1.870378e+24,1.701748e+24,8.335784e+23,3.662915e+23,1.354121e+23


In [40]:
d_data['redshift'] = z.loc[d_data['fullbay'].index]
head(d_data['redshift'])

Shape (4333,)


id
NGC5358       0.009181
NGC3252       0.004562
ESO407-002    0.005403
NGC4194       0.008817
NGC7172       0.007644
Name: redshift, dtype: float64

# Save to pickle file

`d_dfs` will be a central object, where we store a series of dataframes into a dictionary. This object is pickled, and the other notebooks will directly read this pickled file.

In [41]:
with open('./data/d_data.pkl', 'wb') as outf:
    pickle.dump(d_data, outf)